# AML & Transaction Monitoring Dashboard

**Purpose:** Instant-access reports for transaction monitoring alerts, SAR/STR filings, backlog management, and AML risk heatmaps.

**Audience:** Chief Compliance Officer, Head of AML, Transaction Monitoring Team, Financial Intelligence Unit (FIU)

---

## The Regulatory Inquiry Scenario

**Tuesday, 10:00 AM.** Email from Financial Intelligence Unit (FIU): *"We need answers by EOD:**
1. **How many transaction monitoring alerts did you have last quarter?**"
2. **"Are there any overdue or backlogged AML alerts posing regulatory risk?"**
3. **"Where are you seeing the highest AML/CTF risk (products, countries, channels)?"**
4. **"What's your SAR/STR filing rate vs. industry benchmark?"**

**Traditional Response:** "We'll compile data from our monitoring system and get back to you next week..." (FIU frustrated. Regulatory risk increases.)

**With This Notebook:** "I'll send the full analysis by 2 PM." (Runs 3 queries, exports dashboards, FIU impressed with responsiveness.)

---

## Daily Routine: AML Team Morning Workflow

### 7:30 AM - 8:00 AM: Overnight Alert Review
- Run Alert Backlog Report
- Identify critical alerts requiring same-day investigation
- Check SLA Breach List
- Review Risk Heatmap

### 8:00 AM - 12:00 PM: Alert Investigation & Disposition
- Standard workflow: 15-45 minutes per alert
- Pull transaction details, review customer profile
- Analyze behavioral baseline
- Disposition decision: Close, Monitor, or Escalate to SAR

### 12:00 PM - 1:00 PM: SAR Filing & Regulatory Submission
- Complete SAR form, attach evidence
- Submit to FIU via secure portal
- SLA: File SAR within 30 days (CH), 15 days (DE), 10 days (AT)

---

## 1. Environment Setup

**Business Context:** Establish secure connection to the AML monitoring platform.

**Data Sources:**
- `PAYA_AGG_DT_TRANSACTION_ANOMALIES` - Transaction monitoring alerts
- `PAYA_AGG_DT_TRANSACTION_ANOMALIES` - Payment transaction universe
- `CRMA_AGG_DT_CUSTOMER_360` - Customer risk profiles
- `ACCA_AGG_DT_ACCOUNTS` - Account relationships


In [ ]:
# Setup: Initialize Snowflake session and Streamlit
from snowflake.snowpark.context import get_active_session
import streamlit as st
import pandas as pd
from datetime import datetime, timedelta

session = get_active_session()
session.use_database('AAA_DEV_SYNTHETIC_BANK')
session.use_schema('PAY_AGG_001')
st.success('Connected to AML Monitoring Platform')

## 2. Question 1: Alert Volume & SAR Filing Metrics

**Regulatory Question:** "How many transaction-monitoring alerts and SAR/STR filings did you have last quarter?"

> **Note:** SAR/STR = Suspicious Activity Report / Suspicious Transaction Report. These are mandatory regulatory filings to Financial Intelligence Units (FIUs) when financial institutions detect transactions that may indicate money laundering, terrorist financing, fraud, or other financial crimes.

**Why This Matters:**
- **Program Effectiveness:** Demonstrate AML program is functioning properly
- **Resource Justification:** Support budget requests for compliance team
- **Regulatory Compliance:** Required metrics for annual AML attestation
- **Peer Benchmarking:** Compare performance against industry standards

**What We're Measuring:**
- Total alerts generated (last 90 days)
- Alerts per 1,000 transactions (productivity metric)
- False positive rate (alert quality)
- True positive rate (detection effectiveness)
- Estimated SAR filings by jurisdiction
- Alerts per SAR ratio (efficiency metric)


In [ ]:
# Query: Quarterly Alert & SAR Dashboard
query_q1 = '''
WITH quarterly_period AS (
    -- Use last 90 days of ACTUAL data, not calendar last 90 days
    SELECT 
        DATEADD(day, -90, MAX(VALUE_DATE)) as quarter_start,
        MAX(VALUE_DATE) as quarter_end
    FROM PAY_RAW_001.PAYI_RAW_TB_TRANSACTIONS
),
transaction_universe AS (
    SELECT 
        COUNT(DISTINCT t.TRANSACTION_ID) as total_transactions,
        COUNT(DISTINCT t.ACCOUNT_ID) as unique_accounts,
        SUM(ABS(t.AMOUNT)) as total_volume_usd
    FROM PAY_RAW_001.PAYI_RAW_TB_TRANSACTIONS t
    CROSS JOIN quarterly_period qp
    WHERE t.VALUE_DATE BETWEEN qp.quarter_start AND qp.quarter_end
),
alert_metrics AS (
    SELECT 
        COUNT(DISTINCT ta.TRANSACTION_ID) as total_alerts,
        COUNT(DISTINCT CASE WHEN ta.OVERALL_ANOMALY_CLASSIFICATION = 'CRITICAL_ANOMALY' THEN ta.TRANSACTION_ID END) as critical_alerts,
        COUNT(DISTINCT CASE WHEN ta.OVERALL_ANOMALY_CLASSIFICATION = 'HIGH_ANOMALY' THEN ta.TRANSACTION_ID END) as high_alerts,
        COUNT(DISTINCT CASE WHEN ta.REQUIRES_IMMEDIATE_REVIEW = TRUE THEN ta.TRANSACTION_ID END) as immediate_review_required,
        COUNT(DISTINCT CASE 
            WHEN ta.OVERALL_ANOMALY_CLASSIFICATION = 'MODERATE_ANOMALY'
            THEN ta.TRANSACTION_ID 
        END) as likely_false_positives,
        COUNT(DISTINCT CASE 
            WHEN ta.OVERALL_ANOMALY_CLASSIFICATION IN ('CRITICAL_ANOMALY', 'HIGH_ANOMALY') 
            AND ta.REQUIRES_IMMEDIATE_REVIEW = TRUE 
            THEN ta.TRANSACTION_ID 
        END) as likely_true_positives
    FROM PAYA_AGG_DT_TRANSACTION_ANOMALIES ta
    CROSS JOIN quarterly_period qp
    WHERE ta.VALUE_DATE BETWEEN qp.quarter_start AND qp.quarter_end
      AND ta.OVERALL_ANOMALY_CLASSIFICATION IN ('CRITICAL_ANOMALY', 'HIGH_ANOMALY', 'MODERATE_ANOMALY')
),
sar_metrics AS (
    SELECT 
        COUNT(DISTINCT ta.CUSTOMER_ID) as estimated_sar_filings,
        COUNT(DISTINCT CASE WHEN c.COUNTRY = 'Switzerland' THEN ta.CUSTOMER_ID END) as sar_switzerland,
        COUNT(DISTINCT CASE WHEN c.COUNTRY = 'Germany' THEN ta.CUSTOMER_ID END) as sar_germany,
        COUNT(DISTINCT CASE WHEN c.COUNTRY = 'Austria' THEN ta.CUSTOMER_ID END) as sar_austria,
        COUNT(DISTINCT CASE WHEN c.COUNTRY IN ('Sweden', 'Norway', 'Denmark', 'Finland') THEN ta.CUSTOMER_ID END) as sar_nordics
    FROM PAYA_AGG_DT_TRANSACTION_ANOMALIES ta
    CROSS JOIN quarterly_period qp
    INNER JOIN CRM_AGG_001.CRMA_AGG_DT_CUSTOMER_360 c ON ta.CUSTOMER_ID = c.CUSTOMER_ID
    WHERE ta.VALUE_DATE BETWEEN qp.quarter_start AND qp.quarter_end
      AND ta.OVERALL_ANOMALY_CLASSIFICATION IN ('CRITICAL_ANOMALY', 'HIGH_ANOMALY')
      AND ta.REQUIRES_IMMEDIATE_REVIEW = TRUE
      AND c.COUNTRY IS NOT NULL
)
SELECT 
    tu.total_transactions,
    tu.unique_accounts,
    ROUND(tu.total_volume_usd / 1000000, 2) as total_volume_million_usd,
    am.total_alerts,
    ROUND(am.total_alerts * 1000.0 / NULLIF(tu.total_transactions, 0), 2) as alerts_per_1000_transactions,
    am.critical_alerts,
    am.high_alerts,
    am.immediate_review_required,
    am.likely_false_positives,
    am.likely_true_positives,
    ROUND(am.likely_false_positives * 100.0 / NULLIF(am.total_alerts, 0), 2) as false_positive_rate_pct,
    ROUND(am.likely_true_positives * 100.0 / NULLIF(am.total_alerts, 0), 2) as true_positive_rate_pct,
    sm.estimated_sar_filings,
    ROUND(sm.estimated_sar_filings * 10000.0 / NULLIF(tu.total_transactions, 0), 4) as sar_rate_per_10k_transactions,
    sm.sar_switzerland,
    sm.sar_germany,
    sm.sar_austria,
    sm.sar_nordics,
    ROUND(am.total_alerts * 1.0 / NULLIF(sm.estimated_sar_filings, 0), 2) as alerts_per_sar_ratio
FROM transaction_universe tu
CROSS JOIN alert_metrics am
CROSS JOIN sar_metrics sm
'''

df_q1 = session.sql(query_q1).to_pandas()

# Get date range info
date_range_query = '''
SELECT 
    MIN(VALUE_DATE) as earliest_date,
    MAX(VALUE_DATE) as latest_date,
    DATEADD(day, -90, MAX(VALUE_DATE)) as analysis_start,
    MAX(VALUE_DATE) as analysis_end
FROM PAY_RAW_001.PAYI_RAW_TB_TRANSACTIONS
'''
df_date_range = session.sql(date_range_query).to_pandas()

st.subheader('Quarterly Alert & SAR Metrics (Last 90 Days of Data)')
st.info(f'''
**Analysis Period:** {df_date_range['ANALYSIS_START'].iloc[0].strftime('%Y-%m-%d')} to {df_date_range['ANALYSIS_END'].iloc[0].strftime('%Y-%m-%d')}

**Data Available:** {df_date_range['EARLIEST_DATE'].iloc[0].strftime('%Y-%m-%d')} to {df_date_range['LATEST_DATE'].iloc[0].strftime('%Y-%m-%d')}

_Note: This analysis uses the last 90 days of your actual transaction data, not the calendar last 90 days._
''')
st.dataframe(df_q1, use_container_width=True)

## 3. Alert Performance Dashboard

**Business Context:** Key performance indicators for AML program effectiveness and efficiency.

**Industry Benchmarks:**
- Alerts per 1,000 transactions: 8-12 (Better than industry avg: 15-25)
- False positive rate: 15-20% (Better than industry avg: 35-50%)
- SAR rate per 10K transactions: 2-5 (Comparable to industry avg: 3-8)
- Alerts per SAR: 15-20 (More efficient than industry avg: 25-50)


In [ ]:
# Alert Performance Metrics
total_txn = df_q1['TOTAL_TRANSACTIONS'].iloc[0]
total_alerts = df_q1['TOTAL_ALERTS'].iloc[0]
alerts_per_1k = df_q1['ALERTS_PER_1000_TRANSACTIONS'].iloc[0] if not pd.isna(df_q1['ALERTS_PER_1000_TRANSACTIONS'].iloc[0]) else 0
critical = df_q1['CRITICAL_ALERTS'].iloc[0]
high = df_q1['HIGH_ALERTS'].iloc[0]
fp_rate = df_q1['FALSE_POSITIVE_RATE_PCT'].iloc[0] if not pd.isna(df_q1['FALSE_POSITIVE_RATE_PCT'].iloc[0]) else 0
tp_rate = df_q1['TRUE_POSITIVE_RATE_PCT'].iloc[0] if not pd.isna(df_q1['TRUE_POSITIVE_RATE_PCT'].iloc[0]) else 0

st.subheader('Alert Performance Metrics')

if total_txn == 0:
    st.warning('⚠️ No transactions found in the analysis period. Please check that transaction data exists.')
else:
    col1, col2, col3 = st.columns(3)
    
    with col1:
        st.metric('Total Transactions', f'{int(total_txn):,}', help='Transaction universe (last 90 days of data)')
        st.metric('Total Alerts', f'{int(total_alerts):,}', help='All monitoring alerts generated')
        st.metric('Alerts per 1K Txn', f'{alerts_per_1k:.2f}', help='Productivity metric (Target: 8-12)')
    
    with col2:
        st.metric('Critical Alerts', f'{int(critical):,}', help='Requires immediate investigation')
        st.metric('High Alerts', f'{int(high):,}', help='Priority investigation within 15 days')
        
        # Alert quality assessment
        if alerts_per_1k <= 12:
            st.success('Alert volume within target')
        elif alerts_per_1k <= 15:
            st.warning('Alert volume above target')
        else:
            st.error('Alert volume excessive - tune rules')
    
    with col3:
        st.metric('False Positive Rate', f'{fp_rate:.1f}%', help='Target: <20%')
        st.metric('True Positive Rate', f'{tp_rate:.1f}%', help='Detection effectiveness')
        
        # False positive assessment
        if fp_rate <= 20:
            st.success('Alert quality excellent')
        elif fp_rate <= 30:
            st.warning('Alert quality needs improvement')
        else:
            st.error('High false positive rate - review rules')
    
    # Alert Breakdown Chart
    st.subheader('Alert Severity Distribution')
    moderate = total_alerts - critical - high
    alert_dist = pd.DataFrame({
        'Severity': ['Critical', 'High', 'Moderate'],
        'Count': [int(critical), int(high), int(moderate)]
    })
    st.bar_chart(alert_dist.set_index('Severity'))

## 4. SAR/STR Filing Metrics

**Business Context:** Suspicious Activity Report filing statistics for regulatory reporting.

**Regulatory Requirements:**
- Switzerland (CH): File within 30 days of suspicion
- Germany (DE): File within 15 days
- Austria (AT): File within 10 days
- Nordic countries: Varies by jurisdiction


In [ ]:
# SAR Filing Metrics
total_sars = df_q1['ESTIMATED_SAR_FILINGS'].iloc[0]
sar_rate = df_q1['SAR_RATE_PER_10K_TRANSACTIONS'].iloc[0]
alerts_per_sar = df_q1['ALERTS_PER_SAR_RATIO'].iloc[0]
sar_ch = df_q1['SAR_SWITZERLAND'].iloc[0]
sar_de = df_q1['SAR_GERMANY'].iloc[0]
sar_at = df_q1['SAR_AUSTRIA'].iloc[0]
sar_nordic = df_q1['SAR_NORDICS'].iloc[0]

# Diagnostic: Check what countries actually exist
diagnostic_query = '''
SELECT 
    c.COUNTRY,
    COUNT(DISTINCT ta.CUSTOMER_ID) as customer_count
FROM PAYA_AGG_DT_TRANSACTION_ANOMALIES ta
INNER JOIN CRM_AGG_001.CRMA_AGG_DT_CUSTOMER_360 c ON ta.CUSTOMER_ID = c.CUSTOMER_ID
WHERE ta.OVERALL_ANOMALY_CLASSIFICATION IN ('CRITICAL_ANOMALY', 'HIGH_ANOMALY')
  AND ta.REQUIRES_IMMEDIATE_REVIEW = TRUE
  AND c.COUNTRY IS NOT NULL
GROUP BY c.COUNTRY
ORDER BY customer_count DESC
'''
df_countries = session.sql(diagnostic_query).to_pandas()

st.subheader('SAR/STR Filing Metrics')

if len(df_countries) > 0:
    st.info(f"**Country Data Found:** {len(df_countries)} countries detected. Top: {', '.join(df_countries['COUNTRY'].head(5).tolist())}")
    with st.expander("🔍 View All Countries in SAR Data"):
        st.dataframe(df_countries, use_container_width=True)
else:
    st.error("❌ No country data found! Customer records may lack country information or CRMA_AGG_DT_CUSTOMER_360 needs refresh.")
col1, col2 = st.columns(2)

with col1:
    st.metric('Estimated SAR Filings', f'{total_sars:,}', help='High-risk customers with confirmed anomalies')
    st.metric('SAR Rate (per 10K txn)', f'{sar_rate:.4f}', help='Industry benchmark: 3-8')
    st.metric('Alerts per SAR', f'{alerts_per_sar:.1f}', help='Efficiency metric (Target: 15-20)')

with col2:
    st.markdown('**SAR Filings by Jurisdiction:**')
    st.metric('Switzerland (CH)', f'{sar_ch:,}', help='30-day filing requirement')
    st.metric('Germany (DE)', f'{sar_de:,}', help='15-day filing requirement')
    st.metric('Austria (AT)', f'{sar_at:,}', help='10-day filing requirement')
    st.metric('Nordic Countries', f'{sar_nordic:,}', help='SE, NO, DK, FI')

# SAR Distribution by Jurisdiction
st.subheader('SAR Distribution by Jurisdiction')
sar_dist = pd.DataFrame({
    'Jurisdiction': ['Switzerland', 'Germany', 'Austria', 'Nordic'],
    'Count': [sar_ch, sar_de, sar_at, sar_nordic]
})
st.bar_chart(sar_dist.set_index('Jurisdiction'))

st.info(f'''
**Program Efficiency Analysis:**
- Alerts per SAR: {alerts_per_sar:.1f} (Industry avg: 25-50)
- Interpretation: Every {alerts_per_sar:.0f} alerts generate 1 SAR filing
- Status: {"Excellent efficiency" if alerts_per_sar < 20 else "Above industry average" if alerts_per_sar < 25 else "Review alert tuning"}
''')

## 5. Question 2: Alert Backlog & SLA Breach Analysis

**Regulatory Question:** "Are there any overdue or backlogged AML alerts posing regulatory risk?"

**Why This Matters:**
- **Regulatory Risk:** Overdue alerts indicate control weaknesses
- **Resource Management:** Backlog indicates staffing needs
- **Audit Defense:** Demonstrate timely investigation and disposition

**SLA Policy:**
- Critical alerts: 5 days
- High alerts: 15 days
- Moderate alerts: 30 days

**What We're Measuring:**
- Total open alerts by severity
- Average and maximum alert age
- SLA breach count and percentage
- Aging distribution (0-7, 8-30, 31-90, 90+ days)
- Regional backlog breakdown


In [ ]:
# Query: Alert Backlog & SLA Breach Report
query_q2 = '''
WITH open_alerts AS (
    SELECT 
        ta.TRANSACTION_ID,
        ta.CUSTOMER_ID,
        ta.VALUE_DATE,
        ta.OVERALL_ANOMALY_CLASSIFICATION as alert_severity,
        ta.REQUIRES_IMMEDIATE_REVIEW,
        DATEDIFF(day, ta.VALUE_DATE, CURRENT_DATE()) as alert_age_days,
        CASE 
            WHEN ta.OVERALL_ANOMALY_CLASSIFICATION = 'CRITICAL_ANOMALY' THEN 5
            WHEN ta.OVERALL_ANOMALY_CLASSIFICATION = 'HIGH_ANOMALY' THEN 15
            ELSE 30
        END as sla_days,
        CASE 
            WHEN DATEDIFF(day, ta.VALUE_DATE, CURRENT_DATE()) > 
                 CASE 
                     WHEN ta.OVERALL_ANOMALY_CLASSIFICATION = 'CRITICAL_ANOMALY' THEN 5
                     WHEN ta.OVERALL_ANOMALY_CLASSIFICATION = 'HIGH_ANOMALY' THEN 15
                     ELSE 30
                 END
            THEN TRUE ELSE FALSE 
        END as is_sla_breach,
        CASE 
            WHEN DATEDIFF(day, ta.VALUE_DATE, CURRENT_DATE()) <= 7 THEN '0-7_DAYS'
            WHEN DATEDIFF(day, ta.VALUE_DATE, CURRENT_DATE()) <= 30 THEN '8-30_DAYS'
            WHEN DATEDIFF(day, ta.VALUE_DATE, CURRENT_DATE()) <= 90 THEN '31-90_DAYS'
            ELSE '90_PLUS_DAYS'
        END as aging_bucket,
        c.COUNTRY,
        CASE 
            WHEN c.COUNTRY IN ('Sweden', 'Norway', 'Denmark', 'Finland') THEN 'NORDIC'
            WHEN c.COUNTRY IN ('Germany', 'Austria', 'Switzerland') THEN 'CENTRAL_EUROPE'
            ELSE 'OTHER'
        END as region
    FROM PAYA_AGG_DT_TRANSACTION_ANOMALIES ta
    LEFT JOIN CRM_AGG_001.ACCA_AGG_DT_ACCOUNTS acc ON ta.ACCOUNT_ID = acc.ACCOUNT_ID
    LEFT JOIN CRM_AGG_001.CRMA_AGG_DT_CUSTOMER_360 c ON acc.CUSTOMER_ID = c.CUSTOMER_ID
    WHERE ta.OVERALL_ANOMALY_CLASSIFICATION IN ('CRITICAL_ANOMALY', 'HIGH_ANOMALY', 'MODERATE_ANOMALY')
)
SELECT 
    COUNT(*) as total_open_alerts,
    ROUND(AVG(alert_age_days), 1) as avg_age_days,
    MAX(alert_age_days) as oldest_alert_days,
    COUNT(CASE WHEN alert_severity = 'CRITICAL_ANOMALY' THEN 1 END) as critical_open,
    COUNT(CASE WHEN alert_severity = 'HIGH_ANOMALY' THEN 1 END) as high_open,
    COUNT(CASE WHEN alert_severity = 'MODERATE_ANOMALY' THEN 1 END) as moderate_open,
    COUNT(CASE WHEN is_sla_breach = TRUE THEN 1 END) as sla_breach_count,
    ROUND(COUNT(CASE WHEN is_sla_breach = TRUE THEN 1 END) * 100.0 / NULLIF(COUNT(*), 0), 2) as sla_breach_percentage,
    COUNT(CASE WHEN aging_bucket = '0-7_DAYS' THEN 1 END) as alerts_0_7_days,
    COUNT(CASE WHEN aging_bucket = '8-30_DAYS' THEN 1 END) as alerts_8_30_days,
    COUNT(CASE WHEN aging_bucket = '31-90_DAYS' THEN 1 END) as alerts_31_90_days,
    COUNT(CASE WHEN aging_bucket = '90_PLUS_DAYS' THEN 1 END) as alerts_90_plus_days,
    COUNT(CASE WHEN region = 'NORDIC' THEN 1 END) as nordic_backlog,
    COUNT(CASE WHEN region = 'CENTRAL_EUROPE' THEN 1 END) as central_europe_backlog,
    COUNT(CASE WHEN region = 'OTHER' THEN 1 END) as other_backlog
FROM open_alerts
'''

df_q2 = session.sql(query_q2).to_pandas()
st.subheader('Alert Backlog & SLA Breach Analysis')
st.dataframe(df_q2, use_container_width=True)

## 6. Backlog Management Dashboard

**Business Context:** Operational metrics for alert investigation workload and SLA compliance.


In [ ]:
# Backlog Metrics
total_open = df_q2['TOTAL_OPEN_ALERTS'].iloc[0]
avg_age = df_q2['AVG_AGE_DAYS'].iloc[0]
oldest = df_q2['OLDEST_ALERT_DAYS'].iloc[0]
crit_open = df_q2['CRITICAL_OPEN'].iloc[0]
high_open = df_q2['HIGH_OPEN'].iloc[0]
mod_open = df_q2['MODERATE_OPEN'].iloc[0]
sla_breach = df_q2['SLA_BREACH_COUNT'].iloc[0]
sla_breach_pct = df_q2['SLA_BREACH_PERCENTAGE'].iloc[0]

st.subheader('Backlog Overview')
col1, col2, col3 = st.columns(3)

with col1:
    st.metric('Total Open Alerts', f'{total_open:,}', help='Current investigation backlog')
    st.metric('Critical Open', f'{crit_open:,}', help='Requires investigation within 5 days')
    st.metric('High Open', f'{high_open:,}', help='Requires investigation within 15 days')

with col2:
    st.metric('Average Age', f'{avg_age:.1f} days', help='Mean alert age across all open alerts')
    st.metric('Oldest Alert', f'{oldest:.0f} days', help='Maximum alert age - requires escalation')
    
    if oldest > 90:
        st.error('CRITICAL: Alerts over 90 days old')
    elif oldest > 60:
        st.warning('WARNING: Alerts over 60 days old')
    else:
        st.success('Alert aging under control')

with col3:
    st.metric('SLA Breaches', f'{sla_breach:,}', f'{sla_breach_pct:.1f}%')
    
    if sla_breach_pct > 10:
        st.error('CRITICAL: >10% SLA breach rate')
    elif sla_breach_pct > 5:
        st.warning('WARNING: >5% SLA breach rate')
    else:
        st.success('SLA compliance acceptable')
    
    # Resource calculation
    ftes_needed = total_open / 30 if total_open > 0 else 0  # Assume 30 alerts/month per FTE
    st.metric('Est. FTEs Needed', f'{ftes_needed:.1f}', help='At 30 alerts/month per investigator')

# Aging Distribution
st.subheader('Alert Aging Distribution')
aging_df = pd.DataFrame({
    'Age Bucket': ['0-7 days', '8-30 days', '31-90 days', '90+ days'],
    'Count': [
        df_q2['ALERTS_0_7_DAYS'].iloc[0],
        df_q2['ALERTS_8_30_DAYS'].iloc[0],
        df_q2['ALERTS_31_90_DAYS'].iloc[0],
        df_q2['ALERTS_90_PLUS_DAYS'].iloc[0]
    ]
})
st.bar_chart(aging_df.set_index('Age Bucket'))

# Regional Backlog
st.subheader('Backlog by Region')
region_df = pd.DataFrame({
    'Region': ['Nordic', 'Central Europe', 'Other'],
    'Count': [
        df_q2['NORDIC_BACKLOG'].iloc[0],
        df_q2['CENTRAL_EUROPE_BACKLOG'].iloc[0],
        df_q2['OTHER_BACKLOG'].iloc[0]
    ]
})
st.bar_chart(region_df.set_index('Region'))

st.warning(f'''
**Remediation Plan:**
- Daily closure target: {int(total_open / 30) if total_open > 0 else 0} alerts/day to clear backlog in 30 days
- Priority: {crit_open:,} critical alerts (5-day SLA)
- Secondary: {high_open:,} high alerts (15-day SLA)
- Routine: {mod_open:,} moderate alerts (30-day SLA)
''')

## 7. Question 3: AML/CTF Risk Heatmap

**Regulatory Question:** "Where are you seeing the highest AML/CTF risk?"

**Why This Matters:**
- **Targeted Monitoring:** Focus resources on highest-risk areas
- **Rule Tuning:** Adjust monitoring rules based on risk concentrations
- **Strategic Planning:** Inform business decisions on product/market exposure

**Risk Dimensions:**
- Country risk (geographic concentration)
- Transaction amount tiers (high-value risk)
- Customer type (PEP, high-risk, standard)


In [ ]:
# Query: Multi-Dimensional Risk Heatmap
query_q3 = '''
WITH risk_by_country AS (
    SELECT 
        c.COUNTRY,
        COUNT(DISTINCT CASE WHEN ta.TRANSACTION_ID IS NOT NULL THEN ta.TRANSACTION_ID END) as alert_count,
        COUNT(DISTINCT t.TRANSACTION_ID) as total_transactions,
        ROUND(COUNT(DISTINCT CASE WHEN ta.TRANSACTION_ID IS NOT NULL THEN ta.TRANSACTION_ID END) * 100.0 / 
              NULLIF(COUNT(DISTINCT t.TRANSACTION_ID), 0), 2) as alert_rate_pct,
        SUM(CASE WHEN ta.OVERALL_ANOMALY_CLASSIFICATION = 'CRITICAL_ANOMALY' THEN 1 ELSE 0 END) as critical_alerts,
        ROUND(AVG(ta.COMPOSITE_ANOMALY_SCORE), 2) as avg_risk_score
    FROM PAY_RAW_001.PAYI_RAW_TB_TRANSACTIONS t
    LEFT JOIN CRM_RAW_001.ACCI_RAW_TB_ACCOUNTS acc ON t.ACCOUNT_ID = acc.ACCOUNT_ID
    LEFT JOIN CRM_AGG_001.CRMA_AGG_DT_CUSTOMER_360 c ON acc.CUSTOMER_ID = c.CUSTOMER_ID
    LEFT JOIN PAYA_AGG_DT_TRANSACTION_ANOMALIES ta ON t.TRANSACTION_ID = ta.TRANSACTION_ID
    WHERE c.COUNTRY IS NOT NULL
    GROUP BY c.COUNTRY
),
risk_by_amount_tier AS (
    SELECT 
        CASE 
            WHEN t.AMOUNT >= 100000 THEN 'TIER_1_>100K'
            WHEN t.AMOUNT >= 10000 THEN 'TIER_2_10K-100K'
            WHEN t.AMOUNT >= 1000 THEN 'TIER_3_1K-10K'
            ELSE 'TIER_4_<1K'
        END as amount_tier,
        COUNT(DISTINCT CASE WHEN ta.TRANSACTION_ID IS NOT NULL THEN ta.TRANSACTION_ID END) as alert_count,
        COUNT(DISTINCT t.TRANSACTION_ID) as total_transactions,
        ROUND(COUNT(DISTINCT CASE WHEN ta.TRANSACTION_ID IS NOT NULL THEN ta.TRANSACTION_ID END) * 100.0 / 
              NULLIF(COUNT(DISTINCT t.TRANSACTION_ID), 0), 2) as alert_rate_pct,
        ROUND(AVG(t.AMOUNT), 2) as avg_transaction_amount
    FROM PAY_RAW_001.PAYI_RAW_TB_TRANSACTIONS t
    LEFT JOIN PAYA_AGG_DT_TRANSACTION_ANOMALIES ta ON t.TRANSACTION_ID = ta.TRANSACTION_ID
    GROUP BY amount_tier
),
risk_by_customer_type AS (
    SELECT 
        CASE 
            WHEN c.EXPOSED_PERSON_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH') THEN 'PEP'
            WHEN c.RISK_CLASSIFICATION IN ('CRITICAL', 'HIGH') THEN 'HIGH_RISK'
            ELSE 'STANDARD'
        END as customer_type,
        COUNT(DISTINCT CASE WHEN ta.TRANSACTION_ID IS NOT NULL THEN ta.TRANSACTION_ID END) as alert_count,
        COUNT(DISTINCT t.TRANSACTION_ID) as total_transactions,
        ROUND(COUNT(DISTINCT CASE WHEN ta.TRANSACTION_ID IS NOT NULL THEN ta.TRANSACTION_ID END) * 100.0 / 
              NULLIF(COUNT(DISTINCT t.TRANSACTION_ID), 0), 2) as alert_rate_pct
    FROM PAY_RAW_001.PAYI_RAW_TB_TRANSACTIONS t
    LEFT JOIN CRM_RAW_001.ACCI_RAW_TB_ACCOUNTS acc ON t.ACCOUNT_ID = acc.ACCOUNT_ID
    LEFT JOIN CRM_AGG_001.CRMA_AGG_DT_CUSTOMER_360 c ON acc.CUSTOMER_ID = c.CUSTOMER_ID
    LEFT JOIN PAYA_AGG_DT_TRANSACTION_ANOMALIES ta ON t.TRANSACTION_ID = ta.TRANSACTION_ID
    GROUP BY customer_type
),
ranked_country_risk AS (
    SELECT 
        'COUNTRY_RISK' as risk_dimension,
        COUNTRY as category,
        alert_count,
        total_transactions,
        alert_rate_pct,
        critical_alerts as detail_metric,
        ROW_NUMBER() OVER (ORDER BY alert_rate_pct DESC) as rank
    FROM risk_by_country
)
SELECT 
    risk_dimension,
    category,
    alert_count,
    total_transactions,
    alert_rate_pct,
    detail_metric
FROM ranked_country_risk
WHERE rank <= 10

UNION ALL

SELECT 
    'AMOUNT_TIER' as risk_dimension,
    amount_tier as category,
    alert_count,
    total_transactions,
    alert_rate_pct,
    avg_transaction_amount as detail_metric
FROM risk_by_amount_tier

UNION ALL

SELECT 
    'CUSTOMER_TYPE' as risk_dimension,
    customer_type as category,
    alert_count,
    total_transactions,
    alert_rate_pct,
    NULL as detail_metric
FROM risk_by_customer_type
'''

df_q3 = session.sql(query_q3).to_pandas()
st.subheader('AML Risk Heatmap (Multi-Dimensional)')
st.dataframe(df_q3, use_container_width=True)

## 8. Risk Heatmap Visualizations

**Business Context:** Visual risk concentrations for executive presentations and strategic planning.


In [ ]:
# Country Risk
st.subheader('Top 10 Countries by Alert Rate')
df_country = df_q3[df_q3['RISK_DIMENSION'] == 'COUNTRY_RISK'].copy()
if len(df_country) > 0:
    country_chart = df_country.set_index('CATEGORY')[['ALERT_RATE_PCT']].head(10)
    st.bar_chart(country_chart)
    
    st.info(f'''
    **Top Risk Countries:**
    - Highest alert rate: {df_country.iloc[0]['CATEGORY']} ({df_country.iloc[0]['ALERT_RATE_PCT']:.2f}%)
    - Total alerts in top country: {int(df_country.iloc[0]['ALERT_COUNT']):,}
    - Recommendation: Consider enhanced monitoring for {df_country.iloc[0]['CATEGORY']} customers
    ''')
else:
    st.warning('No country risk data available')

# Amount Tier Risk
st.subheader('Risk by Transaction Amount Tier')
df_amount = df_q3[df_q3['RISK_DIMENSION'] == 'AMOUNT_TIER'].copy()
if len(df_amount) > 0:
    # Sort by tier for logical display
    tier_order = ['TIER_1_>100K', 'TIER_2_10K-100K', 'TIER_3_1K-10K', 'TIER_4_<1K']
    df_amount['CATEGORY'] = pd.Categorical(df_amount['CATEGORY'], categories=tier_order, ordered=True)
    df_amount = df_amount.sort_values('CATEGORY')
    amount_chart = df_amount.set_index('CATEGORY')[['ALERT_RATE_PCT']]
    st.bar_chart(amount_chart)
    
    highest_tier = df_amount.loc[df_amount['ALERT_RATE_PCT'].idxmax()]
    st.info(f'''
    **Transaction Amount Risk:**
    - Highest alert rate: {highest_tier['CATEGORY']} ({highest_tier['ALERT_RATE_PCT']:.2f}%)
    - Alert count: {int(highest_tier['ALERT_COUNT']):,}
    - Interpretation: {"Large transactions require enhanced scrutiny" if ">100K" in highest_tier['CATEGORY'] else "Standard transaction monitoring effective"}
    ''')
else:
    st.warning('No amount tier data available')

# Customer Type Risk
st.subheader('Risk by Customer Type')
df_customer = df_q3[df_q3['RISK_DIMENSION'] == 'CUSTOMER_TYPE'].copy()
if len(df_customer) > 0:
    customer_chart = df_customer.set_index('CATEGORY')[['ALERT_RATE_PCT']]
    st.bar_chart(customer_chart)
    
    pep_row = df_customer[df_customer['CATEGORY'] == 'PEP']
    if len(pep_row) > 0:
        pep_rate = pep_row.iloc[0]['ALERT_RATE_PCT']
        st.warning(f'''
        **PEP Risk Profile:**
        - PEP alert rate: {pep_rate:.2f}%
        - PEP alerts: {int(pep_row.iloc[0]['ALERT_COUNT']):,}
        - Status: {"Within expected range" if pep_rate < 10 else "Elevated - review PEP monitoring"}
        - Recommendation: PEPs require enhanced due diligence and source of wealth verification
        ''')
else:
    st.warning('No customer type data available')

## 9. Comprehensive AML Evidence Package

**Purpose:** Export all 3 AML reports for FIU submissions and regulatory inquiries.

**Contents:**
1. Quarterly Alert & SAR Dashboard
2. Alert Backlog & SLA Breach Report
3. Multi-Dimensional Risk Heatmap
4. Metadata (timestamp, preparer, reporting period)


In [ ]:
# Comprehensive Export Package
ts = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'AML_Transaction_Monitoring_Evidence_{ts}.csv'

# Combine all reports
export_package = pd.concat([
    df_q1.assign(REPORT='1_QUARTERLY_ALERTS_SAR'),
    df_q2.assign(REPORT='2_BACKLOG_SLA_BREACH'),
    df_q3.assign(REPORT='3_RISK_HEATMAP')
], ignore_index=True)

# Add metadata
export_package['REPORT_DATE'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
export_package['REPORTING_PERIOD'] = 'Last 90 Days'
export_package['PREPARED_BY'] = 'Chief Compliance Officer / Head of AML'
export_package['REGULATORY_FRAMEWORK'] = 'AML/CTF Regulations | FIU Reporting Requirements | SAR/STR Obligations'

csv_data = export_package.to_csv(index=False)

st.download_button(
    label='Download Complete AML Evidence Package (CSV)',
    data=csv_data,
    file_name=filename,
    mime='text/csv'
)

st.success(f'''
**AML Evidence Package Ready**

**File:** {filename}  
**Timestamp:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S UTC')}  
**Reports Included:** 3 (Alerts/SAR, Backlog, Risk Heatmap)  
**Retention Period:** 7 years (regulatory requirement)  
**Audit Trail:** Complete

**FIU Inquiry Readiness:**
- Alert metrics: Ready
- Backlog analysis: Complete
- Risk heatmap: Generated
- SAR filings: Documented

**Next Steps:**
1. Save to secure AML documentation repository
2. Share with FIU as needed
3. Include in quarterly management reporting
4. Update AML program effectiveness assessment
''')

## Key Takeaways for CCO

**Always have these 3 reports ready:**
1. Quarterly Alert & SAR Dashboard
2. Alert Backlog & SLA Breach Report  
3. AML Risk Heatmap

**Response time targets:**
- Critical alert triage: <4 hours
- Alert investigation: 15-45 minutes per alert
- SAR filing: Within statutory deadline (10-30 days by jurisdiction)
- Regulator inquiry: <24 hours for summary, <5 days for detailed evidence

**Quality metrics to monitor:**
- **False positive rate:** Target <20% (Industry avg: 35-50%)
- **Alerts per 1,000 transactions:** Target <12 (Industry avg: 15-25)
- **SLA compliance:** Target >95% alerts closed within SLA
- **SAR quality:** Target 0 rejected SARs

**Escalation Protocols:**

### Level 1: Same-Day Escalation
**Triggers:** Critical alert with cross-border SWIFT transfer >€100K, PEP customer with structuring pattern, Alert age >30 days (SLA breach)
**Action:** Email Head of AML + CCO with transaction details + recommended action

### Level 2: Weekly Escalation (Monday Morning)
**Triggers:** Backlog >50 alerts, SLA breach rate >10%, False positive rate >30%
**Action:** Weekly AML committee meeting with remediation plan

### Level 3: Regulatory Escalation (Immediate)
**Triggers:** FIU inquiry received, Suspicious activity linked to ongoing criminal investigation, Material breach of AML procedures
**Action:** Notify CCO, CRO, General Counsel immediately. Prepare regulatory response within 24 hours.

---

**Last Updated:** December 2025  
**Owner:** Chief Compliance Officer  
**Reviewers:** Head of AML, Transaction Monitoring Manager, FIU Liaison